In [5]:
import zipfile
import os
import tempfile
import geopandas as gpd
import fiona

def zip_gpkg_to_geojson_with_names(zip_pfad, geojson_pfad):
    # Temporäres Verzeichnis zum Entpacken
    with tempfile.TemporaryDirectory() as tmpdir:
        # ZIP entpacken
        with zipfile.ZipFile(zip_pfad, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)

        # GPKG-Datei finden
        gpkg_datei = None
        for root, dirs, files in os.walk(tmpdir):
            for file in files:
                if file.lower().endswith(".gpkg"):
                    gpkg_datei = os.path.join(root, file)
                    break
            if gpkg_datei:
                break

        if not gpkg_datei:
            raise FileNotFoundError("Keine .gpkg-Datei im ZIP gefunden!")

        print(f"GPKG gefunden: {gpkg_datei}")

        # Alle Layer auflisten
        print("Verfügbare Layer:")
        layerliste = fiona.listlayers(gpkg_datei)
        for layer in layerliste:
            print(f" - {layer}")

        # Richtiger Layer mit Gemeindeflächen ist in der Regel:
        layername = "tlm_hoheitsgebiet"  # So heißt er i.d.R. bei Swisstopo
        if layername not in layerliste:
            raise ValueError(f"Layer '{layername}' nicht im GPKG gefunden!")

        # GPKG-Layer laden
        gdf = gpd.read_file(gpkg_datei, layer=layername)

        # Prüfen auf Gemeindename-Spalte
        name_spalten = [col for col in gdf.columns if "name" in col.lower() or "gn_na1" in col.lower()]
        if not name_spalten:
            raise ValueError("Keine Spalte mit Gemeindenamen gefunden!")
        else:
            print(f"Gefundene Namensspalte(n): {name_spalten}")

        # Falls nötig: Spalten umbenennen
        gdf = gdf.rename(columns={name_spalten[0]: "Gemeindename"})

        # Koordinatensystem auf WGS84 (EPSG:4326) transformieren
        if gdf.crs is None:
            gdf.set_crs(epsg=2056, inplace=True)
        gdf = gdf.to_crs(epsg=4326)

        # GeoJSON schreiben (nur relevante Spalten speichern)
        gdf[["Gemeindename", "geometry"]].to_file(geojson_pfad, driver="GeoJSON")
        print(f"GeoJSON erfolgreich gespeichert unter: {geojson_pfad}")

# Beispielnutzung
zip_pfad = "data/swissboundaries3d_2025-04_2056_5728.gpkg.zip"
geojson_pfad = "data/swissBOUNDARIES3D_1_3_TLM_GEMEINDEGEBIET.geojson"

zip_gpkg_to_geojson_with_names(zip_pfad, geojson_pfad)


GPKG gefunden: C:\Users\sarah\AppData\Local\Temp\tmpn7kej_2u\swissBOUNDARIES3D_1_5_LV95_LN02.gpkg
Verfügbare Layer:
 - tlm_hoheitsgrenze
 - tlm_hoheitsgebiet
 - tlm_bezirksgebiet
 - tlm_landesgebiet
 - tlm_kantonsgebiet
Gefundene Namensspalte(n): ['name']
GeoJSON erfolgreich gespeichert unter: data/swissBOUNDARIES3D_1_3_TLM_GEMEINDEGEBIET.geojson
